# IBM Stock Market Prediction Using LSTM with Pytorch
Ilesingha I.T.S.(IT21206078)

### Libraries and settings

In [14]:
import numpy as np
import pandas as pd 
from pylab import mpl, plt
%matplotlib inline

import os
for dirname, _, filenames in os.walk('/DATA'):
    for i, filename in enumerate(filenames):
        if i<5:
            print(os.path.join(dirname,filename))


### Load data

In [15]:
def stocks_data(symbols, dates):
    df = pd.DataFrame(index=dates)
    for symbol in symbols:
        df_temp = pd.read_csv(r"DATA/{}.csv".format(symbol), index_col='Date',
                parse_dates=True, usecols=['Date', 'Close'], na_values=['nan'])
        df_temp = df_temp.rename(columns={'Close': symbol})
        df = df.join(df_temp)
    return df

In [17]:
df.head()

,ibm
2015-01-02,154.933075
2015-01-05,152.495224
2015-01-06,149.206497
2015-01-07,148.231354
2015-01-08,151.453156
